In [1]:
import torch

# checkpoint = torch.load('/home/surya/Downloads/fpn_dat_t_80k.pth')
checkpoint = torch.load('/home/surya/Downloads/mrcn_dat_t_1x.pth')

from collections import OrderedDict
new_state_dict = OrderedDict()

for k,v in checkpoint['state_dict'].items():
    if('backbone' in k):
        if(k.startswith("backbone.norms.")):
            pass
        else:
            new_key = k.replace("backbone.", "")
            new_state_dict[new_key] = v
torch.save(new_state_dict, '/home/surya/Downloads/mrcn_dat_t_backbone.pth')

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load pretrained model
checkpoint = torch.load('/home/surya/Downloads/mrcn_dat_t_backbone.pth')

from dat import DAT
model = DAT()
model.eval();
model.to(device);
model.load_state_dict(checkpoint, strict=False)

_IncompatibleKeys(missing_keys=['cls_norm.norm.weight', 'cls_norm.norm.bias', 'cls_head.weight', 'cls_head.bias'], unexpected_keys=[])

In [3]:
new_state_dict_keyset = set(new_state_dict.keys())
model_state_dict_keyset = set(model.state_dict().keys())
model_state_dict_keyset.difference(new_state_dict_keyset)

{'cls_head.bias',
 'cls_head.weight',
 'cls_norm.norm.bias',
 'cls_norm.norm.weight'}

In [ ]:
# print(torch.sum(last_stage_attention))
# print(torch.sum(last_stage_attention, dim=2))

In [ ]:
# for i, attn_i in enumerate(output_attns):
#     top_scores, _ = torch.sort(attn_i.flatten(), descending=True)
#     print(i, f"{top_scores}") #  : .4f

In [ ]:
# def calculate_cumulative_attention(attentions):    
#     cumulative_attentions = []
#     if len(attentions) > 0:
#         def process_attention(attn):
#             nheads, npatches, _ = attn.shape
#             normalized_attn = F.softmax(attn.sum(dim =0), dim=1)
#             h = int(npatches **0.5)
#             normalized_attn = normalized_attn.reshape(h, h, -1)
#             return normalized_attn

#         def scale_attention(attn, feat_size, nheads):
#             attn = attn.unsqueeze(0)
#             if((attn.shape[0] == feat_size) and (attn.shape[1] == feat_size)):
#                 scaled_attn = attn
#             else:
#                 attn = einops.rearrange(attn, 'bs h w k -> bs k h w')
#                 scaled_attn = F.interpolate(attn, (feat_size, feat_size), 
#                                           mode='bilinear', align_corners=True)
#             scaled_attn = einops.rearrange(scaled_attn, 'bs k h w -> bs (h w) k')
#             scaled_attn = scaled_attn.expand(nheads, -1, -1)
#             return scaled_attn

#         # start from last stage
#         cumulative_attentions.append(attentions[-1])
#         for i in range(1, len(attentions)):
#             prev_attn = cumulative_attentions[-1]
#             prev_attn = process_attention(prev_attn)
            
#             current_attn = attentions[len(attentions) - i - 1]
#             nheads, npatches, _ = current_attn.shape
#             curr_feat_size = int(npatches ** 0.5)
#             scaled_prev_attn = scale_attention(prev_attn, curr_feat_size, nheads)
#             current_attn = current_attn * scaled_prev_attn
#             current_attn = F.softmax(current_attn, dim=2)
#             cumulative_attentions.append(current_attn)

#     # we stored in reverse order
#     cumulative_attentions = cumulative_attentions[::-1]
#     return cumulative_attentions

In [ ]:
# cumulative_attn = calculate_cumulative_attention(output_attns)

In [ ]:
# for i, attn_i in enumerate(cumulative_attn):
#     top_scores, _ = torch.sort(attn_i.flatten(), descending=True)
#     print(i, f"{top_scores}") #  : .4f

In [ ]:
# take current attention map = [nh_o np_o 49]
# interpolate pre attn map to current resolution = [np_o 49]
# repeat scaled map to nh_o = [nh_o np_o 49]
# apply softmax across 49 = [nh_o np_o 49]
# set current stage to previous_stage            

# prev stage attention = [nh, np, 49]
# sum across heads = [np, 49]
# normalize against 49 = [np, 49]
# reshape to height, width = [hi wi 49]